In [1]:
import csv
# remove this and move to just argparse -- use only standard modules
import ArgConfigParse
from dateutil import rrule
from datetime import datetime
from datetime import timedelta
import logging
from pathlib import Path
import json
import argparse
from sys import exit
from sys import argv

In [2]:
WEEKDAYS = {
    'MONDAY': 0,
    'TUESDAY': 1,
    'WEDNESDAY': 2,
    'THURSDAY': 3,
    'FRIDAY': 4,
    'SATURDAY': 5,
    'SUNDAY': 6
}

In [3]:
def do_exit(msg=None, level=0):
    if level > 0:
        
        print(f'exiting due to error:\n     {msg}')
    elif msg:
        print(f'{msg}')
        print('exiting')
    exit(level)
    

In [4]:
def build_empty_schedule(days, blocks):
    '''build a JSON compatible dictionary with all required schedule fields
    Args:
        days(`int`): number of days in a cycle
        blocks(`int`): total number of blocks in a day (including breaks, lunch, etc.)
        
    Returns:
        dict'''
    template_day = {}
    blocks_list = []
    block = {'name': '', 
              'start': '00:00',
              'duration': 0}
    for j in range(0, blocks):
        blocks_list.append(block)

    for i in range(0, days):
        template_day[f'day_{i+1}'] = blocks_list
    return {'standard': template_day, 'alternate': template_day}

In [36]:
def write_empty_schedule(file, days, blocks):
    '''write empty JSON formatted schedule to `file`
    Args:
        file(`Path`): path to write
        days(`int`): number of days in a cycle
        blocks(`int`): total number of blocks in a day including breaks, lunch, etc.
        
    Returns:
        `Path`'''
    
    if Path(file).exists():
        raise FileExistsError(f'{file} already exits, refusing to over-write')
#     json_data = json.dumps(build_empty_schedule())
    with open(file, 'w') as outfile:
        json.dump(build_empty_schedule(days, blocks), outfile, indent=5)
    return file

In [63]:
def read_non_instruction(file, dt_format):
    '''return list of daytime objects based on dates in file
    
    Args:
        file(`str` or `Path`): file to read'''
    # read vacation/non instructional days list
    with open(file, 'r') as open_file:
        file_txt = open_file.readlines()

    non_instruction_dt = []
    for i in file_txt:
        try:
            non_instruction_dt.append(datetime.strptime(i.strip(), dt_format))
        except ValueError as e:
            logging.debug(f'skipping unknown date format in {file_txt}: "{i}""')
        
    return non_instruction_dt

    

In [38]:
def read_schedule_json(file):
    '''read a json formatted file
    Args:
        file(`Path`): file to read
    
    Returns:
        `dict`'''
    try:
        with open(file, 'r') as json_file:
            json_data = json.load(json_file)
    except Exception as e:
        do_exit(f'failed to read json file "{file}": {e}')
    
    return json_data


In [39]:
def set_school_days(start, end, vacation, dt_format):
    '''create a list of school days excluding weekends and non-instructional days
    Args:
        start(`str`): first day of school - in dt_format (e.g. 2021/08/17)
        end(`str`): last day of school - in dt_format (e.g. 2022/06/18)
        vacation(`list`): list of non-instructional days in dt_format
        dt_format(`str`): datetime format e.g. %Y/%m/%d
        
    Returns:
        `list`
        '''
    
    start_dt = datetime.strptime(start, dt_format)
    end_dt = datetime.strptime(end, dt_format)
    
    # set actual school days
    school_days = []
    for dt in rrule.rrule(rrule.DAILY, dtstart=start_dt, until=end_dt):
        if dt not in vacation and datetime.weekday(dt) in range(0, 5):
                school_days.append(dt)
                
    return school_days
        

In [40]:
def get_events(dicts):
    '''process list of of dicts and return all unique `name` values
    Args:
        dicts(`list` of `dict`): list of dictionaries containing key "name"
        
    Returns:
        `set` of name'''
    if not isinstance(dicts, list):
        raise TypeError(f'expected list of dictionaries, recieved {type(dicts)}')
    events_unique = set()
    for d in dicts:
        for day, schedule in d.items():
            for event in schedule:
                events_unique.add(event['name'])
    return(events_unique)

In [41]:
def day_schedule(l, date):
    '''return a google calendar compatible list of dict for writing CSVs
    
    Args:
        l(`list`): list of all blocks from a single day
        date(`str`): date string e.g. 2021/08/17
        
    Returns:
        `list of dict` '''
    dt_format = '%H:%M'
    schedule = []
    for idx, val in enumerate(l):
        subject = val['name']
        start_date = date
        start_time = val['start']
        start_dt = datetime.strptime(start_time, dt_format)
        end_dt = start_dt + timedelta(minutes = val['duration'])
        end_time = datetime.strftime(end_dt, dt_format)
        event = {
            'Subject': subject,
            'Start Date': start_date,
            'Start Time': start_time,
            'End Time': end_time
        }
        schedule.append(event)
    return schedule

In [42]:
def write_csv(filename, title, events):
    '''write a csv file for all events that match "title"
    
    Args: 
        filename(`Path`): file to write
        title(`str`): string to match in event Subject e.g. "Block C"
        events(`list` of `dict`)
        
    Returns: 
        `bool`: true on success
    '''
    with open(filename, 'w') as csvfile:
        fieldnames = events[0].keys()
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        writer.writeheader()
        for event in events:
            if title == '*':
                writer.writerow(event)
            elif title in event['Subject']:
                writer.writerow(event)
                
    return True 

In [12]:
argv

['/Users/aaronciuffo/.local/share/virtualenvs/calendar_csv-LU1jQNan/lib/python3.9/site-packages/ipykernel_launcher.py',
 '-f',
 '/Users/aaronciuffo/Library/Jupyter/runtime/kernel-b115072c-f406-431c-98da-9f9e64206fbf.json']

In [13]:
argv.extend(['-s', '2021/08/17'])

In [14]:
argv.extend(['-e', '2022/06/16'])

In [82]:
argv.extend(['-n', './vacation_days.txt'])


In [90]:
argv.extend(['-c', './schedule.json', '-a', 'Wednesday'])

In [83]:
argv.extend(['-v', '-v'])

In [85]:
argv.pop()


'-v'

In [76]:
def get_args():
    parser = argparse.ArgumentParser(description='process command line arguments')
    parser.add_argument('-s', '--start_date', default=None,
                       help='First day of classes in YYYY/MM/DD format', metavar='"YYYY/MM/DD"')
    parser.add_argument('-e', '--end_date', default=None,
                        help='Last day of classes in YYYY/MM/DD format', metavar='"YYYY/MM/DD"')
    parser.add_argument('-n', '--non_instruction', default=None,
                       help='File containing non-instructional days between start and end date, one per line matching the daytime format (YYYY/MM/DD)')
    parser.add_argument('-c', '--schedule_file', default=None,
                        help='file containing JSON schedule data')
    parser.add_argument('-d', '--date_format', default='%Y/%m/%d',
                       help='datetime format see: https://docs.python.org/3/library/datetime.html#strftime-and-strptime-behavior',
                       metavar='"%Y/%m/%d"')
    parser.add_argument('-a', '--alternate_day', default=None,
                       help='day to use "alternate" schedule', metavar="Wednesday")
    parser.add_argument('-o', '--output_path', default='~/Desktop/',
                       help='location to output CSV files')
    parser.add_argument('-b', '--blank_schedule', default=None,
                       help='generate a blank JSON schedule file template on the Desktop using supplied name',
                       metavar='filename')
    parser.add_argument('-v', '--verbose', action='count', default=0,
                       help='increase verbosity in logging (additional -v increases level)')
    
    return parser.parse_known_args()



In [69]:
args

Namespace(start_date='2021/08/17', end_date='2022/06/16', non_instruction=None, schedule_file=None, date_format='%Y/%m/%d', alternate_day=None, output_path='~/Desktop/', blank_schedule=None, verbose=0)

In [102]:
# def main():
args, unknown_args = get_args()
log_level = 40
if args.verbose:
    log_level = log_level-args.verbose
logging.root.setLevel(log_level)

if args.blank_schedule:
    try:
        write_empty_schedule(Path(args.blank_schedule+'.JSON').expanduser(), 8, 8)
        do_exit(f'wrote blank schedule to {file}.JSON')
    except (FileExistsError, OSError) as e:
        do_exit(e, 1)
        
        
if args.non_instruction:
    file_non_instruction = Path(args.non_instruction).expanduser()  
else:
    do_exit('no non-instructional days file provided; cannot continue')  
try:  
    non_instruction = read_non_instruction(file_non_instruction, dt_format=args.date_format)
except OSError as e:
    do_exit(e, 1)
    
if args.schedule_file:
    file_schedule = Path(args.schedule_file).expanduser()
else:
    do_exit('no schedule file provided; cannot continue')
try:
    schedule_json = read_schedule_json(file_schedule)
except OSError as e:
    do_exit(e, 1)


if args.output_path:
    path_output = Path(args.output_path).expanduser()
else:
    do_exit('no output path supplied; cannot continue')
    
if args.alternate_day:
    try:
        alternate_day = WEEKDAYS[str(args.alternate_day).upper()]
    except KeyError:
        do_exit(f'ERROR: {args.alternate_day} is not a valid alternate day. Valid choices are: {WEEKDAYS.keys()}')
else:
    alternate_day = None

if args.start_date:
    sy_start = args.start_date
else:
    do_exit(f'no school-year start date provided; cannot continue')

if args.end_date:
    sy_end = args.end_date
else:
    do_exit(f'no school-year end date provided; cannot continue ')

if not args.date_format:
    do_exit('no date format provided; cannot continue. HINT: try removing the -d/--date_format arguments')



In [22]:
# # def main():



# config = ArgConfigParse.ConfigFile(['setup.ini'])
# vacation_file = Path('./vacation_days.txt').absolute().expanduser()
# schedule_file = Path('./schedule.json')
# output_path = Path('~/Desktop').expanduser()
# alternate_day = 'Wednesday'

# # parse config files
# config.parse_config()
# main_cfg = config.config_dict

# # wrap in try -- don't require an alternate day (e.g. elementary school)
# alternate_day = WEEKDAYS[alternate_day.upper()]

# sy_start = main_cfg['dates']['start']
# sy_end = main_cfg['dates']['end']
# dt_format = main_cfg['main']['dt_format']

# schedule_json = read_schedule_json(schedule_file)
# schedule_standard = schedule_json['standard']
# rot_len = len(schedule_standard)
# schedule_alt = schedule_json['alternate']
# unique_events = get_events([schedule_standard, schedule_alt])
# vacation = read_vacations(vacation_file)
# school_days = set_school_days(start=sy_start, 
#                               end=sy_end, vacation=vacation, 
#                               dt_format=dt_format)
# # also produce a calendar of just Day 1, day 2, day 3 etc.

# # build a list of all events
# all_events = []
# schedule_lookup = schedule_standard
# for idx, val in enumerate(school_days):
#     # use alternate day schedule
#     if datetime.weekday(val) == alternate_day:
#         schedule_lookup = schedule_alt
#     else:
#         schedule_lookup = schedule_standard
#     # convert date to human readable 
#     date = datetime.strftime(val, dt_format)
#     # use day 1, day 2, day N... schedule
#     day = sorted(schedule_lookup)[idx%rot_len]
#     # add the appropriate schedule for this_day
#     this_day = day_schedule(schedule_lookup[day], date)
#     all_events.extend(this_day)

# # write out a CSV for each unique schedule
# for event in unique_events:
#     output_file = Path(output_path/f'{event}.csv')
#     write_csv(output_file, event, all_events)
# write_csv(output_path/'all events.csv', '*', all_events)

IndentationError: expected an indented block (<ipython-input-22-7750b753de30>, line 15)

In [ ]:
# # also produce a calendar of just Day 1, day 2, day 3 etc.
# all_events = []
# schedule_lookup = schedule_standard
# for idx, val in enumerate(school_days):
#     if datetime.weekday(val) == alternate_day:
#         schedule_lookup = schedule_alt
#     else:
#         schedule_lookup = schedule_standard
#     date = datetime.strftime(val, dt_format)
#     day = sorted(schedule_lookup)[idx%rot_len]
#     this_day = day_schedule(schedule_lookup[day], date)
#     all_events.extend(this_day)

In [ ]:
if __name__ == '__main__':
    s = main()

In [ ]:
# def parse_schedules(schedule_dict):
#     # parse schedule file into lookup tables
#     if not isinstance(schedule_dict, dict):
#         raise TypeError (f'schedule_dict must be of type(dict), not {type(schedule_dict)}')
#     schedule_mttf = {} 
#     schedule_w = {}
#     for key, value in schedule_dict.items():
#         if key.startswith('%'):
#             if 'wednesday' in key:
#                 schedule_w[key] = value
#             else:
#                 schedule_mttf[key] = value
#         else:
#             logging.warning(f'unrecongized "day" item in confiuration file(s) {schedule.config_files}: {key}')
#             logging.info(f'all "day" items must follow the format "[%day N] or "[%day N wednesday]"') 
#     return(schedule_mttf, schedule_w)